# 第４章 形態素解析

## 30.形態素解析結果の読み込み

In [ ]:
import MeCab
with open("neko.txt","r") as f,open("neko.txt.mecab","w") as outfile:
    mecab = MeCab.Tagger()
    outfile.write(mecab.parse(f.read()))

In [ ]:
def neko_lines():
    with open("neko.txt.mecab","r") as neko:
        mors=[]
        for line in neko:
            col = line.split("\t")
            if len(col)>=2:
                res_col = col[1].split(",")
                morphese = {
                    'surface' :col[0],
                    'base': res_col[6],
                    'pos': res_col[0],
                    'pos1': res_col[1]
                }
                mors.append(morphese)
                if res_col[1] == '句点':
                    yield mors
                    mors =[]
                    
lines = neko_lines()
for eru in lines:
    print(eru)

## 31.動詞

In [ ]:
list_v =[]
lines = neko_lines()
for line in lines:
    for oneword in line:
        if oneword['pos'] == "動詞":
            list_v.append(oneword['surface'])
            
print(list_v)

## 32.動詞の原型

In [ ]:
list_v_base = []
lines = neko_lines()
for line in lines:
    for word in line:
        if word['pos'] == '動詞':
            list_v_base.append(word['base'])
            
print(list_v_base)

## 33.AのB

In [ ]:
meisiku = []
lines = neko_lines()
for line in lines:
    for word in line:
        if word['pos'] == '名詞':
            meisiku.append(word['surface'])
            if len(meisiku) == 2:
                meisiku =[]
            if len(meisiku) == 3:
                print(meisiku)
                meisiku =[]
            continue
        if word['surface'] == 'の' and word['pos'] == '助詞' and len(meisiku) == 1:
            meisiku.append(word['surface'])
            continue
        meisiku = []

## 34.名詞の連接

In [ ]:
meisi_list_all = []
meisi_list = []
lines = neko_lines()
for line in lines:
    for word in line:
        if word['pos'] == '名詞':
            meisi_list.append(word['surface'])
            continue
        if len(meisi_list) > 1:
            meisi_list_all.append(meisi_list)
        meisi_list = []
        
        
for ln in meisi_list_all:
    print(ln)

## 35.単語の出現頻度

In [ ]:
def frequency():
    dic = {}
    lines = neko_lines()
    for line in lines:
        for word in line:
            if word['base'] not in dic:
                dic[word['base']] = 1
            else:
                dic[word['base']] += 1
    dic2 = sorted(dic.items(), key=lambda x:x[1],reverse=True)
    return dic2
    
frequency()

## 36.頻度上位10語

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
xaxis = []
yaxis = []
fre10 = frequency()[0:10]
for word in fre10:
    xaxis.append(word[0])
    yaxis.append(word[1])
    
print(xaxis)
print(yaxis)
fp = FontProperties(fname = '/system/Library/Fonts/AquaKana.ttc')
plt.bar(xaxis,yaxis,align='center')
plt.title('頻度上位１０',fontproperties = fp)
plt.xlabel('出現頻度が高い10語',fontproperties = fp)
plt.ylabel('出現頻度',fontproperties = fp)
plt.grid(axis='y')
plt.show()

## 37.猫と共起頻度の高い上位10語

In [ ]:
dic = {}
def add_dic(taked):
    if taked not in dic:
        dic[taked] = 1
    else:
        dic[taked] += 1
        
lines = neko_lines()
for line in lines:
    for neko_idx,word in enumerate(line):
        if word['surface'] =='猫':
            add_dic(line[neko_idx-1]['surface'])
            add_dic(line[neko_idx+1]['surface'])

dic2 = sorted(dic.items(), key=lambda x:x[1],reverse=True)
near_nyan = dic2[:10]
print(near_nyan)
##################################
xaxis =[]
yaxis =[]
for string in near_nyan:
    xaxis.append(string[0])
    yaxis.append(string[1])
fp = FontProperties(fname = '/system/Library/Fonts/AquaKana.ttc')
plt.bar(xaxis,yaxis,align='center')
plt.title('にゃーん',fontproperties = fp)
plt.xlabel('出現頻度が高い10語',fontproperties = fp)
plt.ylabel('出現頻度',fontproperties = fp)
plt.grid(axis='y')
plt.show()

## 38.ヒストグラム

In [ ]:
xaxis = []
yaxis = []
word_list = frequency()
for word in word_list:
    xaxis.append(word[1])
    yaxis.append(word[0])

    
#print(xaxis)
plt.hist(xaxis,bins=20,range=(1,20))
plt.xlim(xmin=1,xmax=20)
plt.grid(axis='y')
plt.show()

## 39.Zipfの法則

In [ ]:
word_list = frequency()
xaxis =[]
yaxis =[]
for rank,word in enumerate(word_list):
    xaxis.append(rank+1)
    yaxis.append(word[1])
    
plt.scatter(range(1,len(yaxis)+1),yaxis)
plt.xlim(1,len(yaxis)+1)
plt.ylim(1,yaxis[0])
plt.xscale('log')
plt.yscale('log')
fp = FontProperties(fname = '/system/Library/Fonts/AquaKana.ttc')
plt.title("39.Zipfの法則",fontproperties=fp)
plt.xlabel('出現順位',fontproperties=fp)
plt.ylabel('出現頻度',fontproperties=fp)
plt.grid(axis='both')
plt.show()